In [1]:
from requests import get, post, delete
import pandas as pd
from sensitive import *

In [35]:
auth = f"Bearer {AUTH_TOKEN}"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


In [3]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    song_artists = ""
    try:
        song_artists = song["track"]["artists"]
    except Exception:
        song_artists = song["artists"]

    for artist in song_artists:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            try:
                artist_genres = artist_genres_res.json()["genres"]
            except Exception:
                print(artist_genres_res.json())
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [4]:
total_song_count = get_total_song_count()

In [6]:


all_genres = []
for offset in range(0, total_song_count, 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [7]:
songs = songs[-total_song_count:]

In [8]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [9]:
playlist


,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"[Justin Bieber, Dominic Fike]",57,"[canadian pop, pop, alternative pop rock]"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"[Kina, Snøw]",83,"[sad lo-fi, sad rap]"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,[Alessia Cara],77,"[canadian contemporary r&b, canadian pop, danc..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"[Teqkoi, Snøw]",69,"[lo-fi chill, sad lo-fi, sad rap]"
4,3xMTM729pJSwUv55WwWcm2,change,[One Hope],44,[sad lo-fi]
...,...,...,...,...,...
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,[push baby],53,[uk pop]
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",[iamnotshane],49,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,[Rory Webley],65,"[alt z, social media pop]"
984,5ARrWiDDRDocvURbemcnCy,Cut My Fingers Off,[Ethan Bortnick],50,[]


In [10]:
playlist.to_csv('mid-term-playlist.csv', header=True, index=None)

In [11]:
all_artists = list(artists.keys())
playlist = pd.read_csv('mid-term-playlist.csv')
playlist

,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"['Justin Bieber', 'Dominic Fike']",57,"['canadian pop', 'pop', 'alternative pop rock']"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"['Kina', 'Snøw']",83,"['sad lo-fi', 'sad rap']"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,['Alessia Cara'],77,"['canadian contemporary r&b', 'canadian pop', ..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"['Teqkoi', 'Snøw']",69,"['lo-fi chill', 'sad lo-fi', 'sad rap']"
4,3xMTM729pJSwUv55WwWcm2,change,['One Hope'],44,['sad lo-fi']
...,...,...,...,...,...
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,['push baby'],53,['uk pop']
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",['iamnotshane'],49,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,['Rory Webley'],65,"['alt z', 'social media pop']"
984,5ARrWiDDRDocvURbemcnCy,Cut My Fingers Off,['Ethan Bortnick'],50,[]


In [12]:
def genres_indexed(genres):
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists):
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [13]:
playlist["genres_indexed"] = [genres_indexed(
    eval(playlist["genres"][x])) for x in range(len(playlist["genres"]))]


In [14]:
playlist.columns = playlist.columns.map(lambda x: x.strip())

In [15]:
playlist["artists_indexed"] = [artists_indexed(
    eval(playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [16]:
playlist.columns


Index(['id', 'name', 'artists', 'popularity', 'genres', 'genres_indexed',
       'artists_indexed'],
      dtype='object')

## KNN


In [17]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [18]:
song_dict = knn_prepared_data(playlist)

In [19]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


In [20]:
def list_distance(a, b):
    distance = 0
    for item_a, item_b in list(zip(a, b)):
        distance += 0 if item_a == item_b else 0.2 if (len(a) < 400) else 0.1

    return distance

In [21]:
def ComputeDistance(a, b):
    genres_distance = list_distance(a['genres_indexed'], b['genres_indexed'])
    artists_distance = list_distance(a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    
    return genres_distance * 1.3 + artists_distance * 0.2 + popularity_distance * 0.004


## Song options

### Choose one of these to get the 50 closest songs to it


In [22]:
def get_genres(genres):
    all_genres = genres[0][:]

    for index in range(1, len(genres)):
        for i in range(0, len(all_genres)):
            all_genres[i] = all_genres[i] or genres[index][i]

    return all_genres

    
def get_artists(artists):
    all_artists = artists[0][:]

    for index in range(1, len(artists)):
        for i in range(0, len(all_artists)):
            all_artists[i] = all_artists[i] or artists[index][i]

    return all_artists

In [23]:
from functools import reduce

top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=medium_term&limit=5', headers=headers).json()
top_5_songs = list(filter(lambda song: song['name'] in list(playlist['name']), list(map(lambda song: {'name': song['name'], 'genres': get_song_genres(song), 'artists': list(map(lambda artist: artist['name'],song['artists'])), 'popularity': song['popularity']},top_5['items']))))

In [39]:
list(map(lambda x: x['name'], top_5['items']))

['The Story of the Paper Boy',
 'thor',
 'A Song For Kaytlyn',
 'PPL ALWAYS MAKE SHT FKN WORSE',
 'House of Cards']

In [24]:
temp_genres = list(reduce(lambda acc, x: acc + list(set(x['genres']) - set(acc)), top_5_songs, []))
temp_artists = list(reduce(lambda acc, x: acc + list(set(x['artists']) - set(acc)), top_5_songs, []))
latest_fav = {'id': "", 'name': "Latest Favorites", 'genres': temp_genres, 'artists': temp_artists}

latest_fav['genres_indexed'] = get_genres(list(map(lambda song: genres_indexed(song['genres']), top_5_songs)))

latest_fav['artists_indexed'] = get_artists(list(map(lambda song: artists_indexed(song['artists']), top_5_songs)))

latest_fav['popularity'] = int(reduce(lambda acc, song: acc + int(song['popularity']), top_5_songs, 0) / len(top_5_songs))


In [25]:
if len(song_dict) == len(songs):
    song_dict.append(latest_fav)
elif len(song_dict) == len(songs) + 1:
    song_dict.pop()
    song_dict.append(latest_fav)
else:
    raise Exception("error in lists indexes")

In [29]:

K = 100
index = len(song_dict) - 1
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'], '\n',
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor]["name"]} \t {song_dict[neighbor]["artists"]} \t {song_dict[neighbor]["genres"]} \t {song_dict[neighbor]["popularity"]}')


Latest Favorites ['Powfu', 'push baby', 'Snøw', 'Skinny Atlas', 'Days to Waste', 'sadeyes', 'Alexander Stewart'] 
 ['sad rap', 'uk pop', 'sad lo-fi', 'emo rap', 'lo-fi chill', 'alt z', 'social media pop', 'post-teen pop'] 48
1. Remains 	 ['Snøw', 'Skinny Atlas'] 	 ['sad lo-fi', 'sad rap', 'emo rap', 'lo-fi chill'] 	 39
2. this is bs. 	 ['Skinny Atlas', 'Laeland', 'Snøw'] 	 ['emo rap', 'lo-fi chill', 'sad lo-fi', 'sad rap'] 	 53
3. future diary (feat. Jomie & Skinny Atlas) 	 ['Powfu', 'Jomie', 'Skinny Atlas'] 	 ['sad rap', 'lo-fi chill', 'sad lo-fi', 'emo rap'] 	 53
4. A Song For Kaytlyn 	 ['Snøw', 'Skinny Atlas'] 	 ['sad lo-fi', 'sad rap', 'emo rap', 'lo-fi chill'] 	 33
5. Evil Queen 	 ['Snøw', 'Skinny Atlas'] 	 ['sad lo-fi', 'sad rap', 'emo rap', 'lo-fi chill'] 	 32
6. summer nights 	 ['Skinny Atlas', 'Laeland', 'Snøw'] 	 ['emo rap', 'lo-fi chill', 'sad lo-fi', 'sad rap'] 	 63
7. Meant to Be 	 ['Skinny Atlas', 'Snøw', 'Jomie', 'Thomas Reid'] 	 ['emo rap', 'lo-fi chill', 'sad lo-fi', '

In [ ]:
def playlist_exists(name):
    request = get('https://api.spotify.com/v1/me/playlists', headers=headers).json()

    playlists = list(map(lambda playlist: (playlist['id'], playlist['name']), request['items']))

    for playlist in playlists:
        if playlist[1] == name:
            return playlist[0]
        
    return False

In [36]:
import json
playlist_name = 'Recent-ish Favorites'
new_id = ""
playlist_id_found = playlist_exists(playlist_name)
if playlist_id_found:
    new_id = playlist_id_found
    
    playlist_tracks = list(map(lambda track: {'uri': track['track']['uri']}, get(f'https://api.spotify.com/v1/playlists/{new_id}/tracks', headers=headers).json()['items']))

    delete_json = delete(f'https://api.spotify.com/v1/playlists/{new_id}/tracks',  headers=headers, data=json.dumps({"tracks": playlist_tracks})).json()
    print(delete_json)
else:
    data = {"name": playlist_name, "description": f"Songs related to your medium term top 5", "public": False}
    playlist_creation = post(f'https://api.spotify.com/v1/users/{USER_ID}/playlists', headers=headers, data=json.dumps(data))
    new_id = playlist_creation.json()['id']

In [37]:
song_uris = ''
for neighbor in neighbors:
    if song_dict[neighbor]["id"] != '':
        song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'
    else:
        continue
song_uris = song_uris[1:]
add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

{'snapshot_id': 'Myw2NTJmYzc4YzBmYzJiYTQ2NTRlYWZlMmQyYzNjNjM3MTdjOTI1OWNh'}